In [1]:
import logging
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import sys
import time
import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
from torchvision.datasets import SVHN
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.utils.data.dataset import TensorDataset

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# !rm -rf /content/backprop/

In [4]:
ff_type = 'backprop'

# dataset = 'MNIST'
# dataset = 'FashionMNIST'
dataset = 'CIFAR10'
#dataset = 'CIFAR100'
# dataset = 'SVHN'

In [5]:
"""
if not os.path.exists(ff_type):
  os.mkdir(ff_type)
        
path = os.path.join(ff_type, dataset)
if not os.path.exists(path):
  os.mkdir(path)
"""

'\nif not os.path.exists(ff_type):\n  os.mkdir(ff_type)\n        \npath = os.path.join(ff_type, dataset)\nif not os.path.exists(path):\n  os.mkdir(path)\n'

In [6]:
"""
def start_logging(logger):
    print("******* nvidia-smi logging started *******")
    while True:
        nvidia_smi_entry = os.popen("nvidia-smi --query-gpu=timestamp,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.free,memory.total,power.draw --format=csv | sed -n '2 p'").read()
        logger.info(nvidia_smi_entry)
    print("******* nvidia-smi logging ended *******")
"""

'\ndef start_logging(logger):\n    print("******* nvidia-smi logging started *******")\n    while True:\n        nvidia_smi_entry = os.popen("nvidia-smi --query-gpu=timestamp,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.free,memory.total,power.draw --format=csv | sed -n \'2 p\'").read()\n        logger.info(nvidia_smi_entry)\n    print("******* nvidia-smi logging ended *******")\n'

In [7]:
'''
formatter = logging.Formatter('%(message)s')
def setup_logger(name, log_file, level=logging.INFO):
    """To setup as many loggers as you want"""

    handler = logging.FileHandler(log_file)
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    
    return logger
'''

'\nformatter = logging.Formatter(\'%(message)s\')\ndef setup_logger(name, log_file, level=logging.INFO):\n    """To setup as many loggers as you want"""\n\n    handler = logging.FileHandler(log_file)\n    handler.setFormatter(formatter)\n\n    logger = logging.getLogger(name)\n    logger.setLevel(level)\n    logger.addHandler(handler)\n    \n    return logger\n'

In [8]:
"""
# nvidia smi logger

logger_nvidia_smi = setup_logger(path+"_logger_nvidia_smi", os.path.join(path, "nvidia_smi.csv"))
system_logger = multiprocessing.Process(target=start_logging, args=[logger_nvidia_smi])

# layer loggers

logger_layer_ff = setup_logger(path+"_logger_layer_ff", os.path.join(path, "layer_ff.csv"))
logger_layer_ff.info('layer_num,layertime_ms')

# e2e loggers

logger_e2e_ff = setup_logger(path+"_logger_e2e_ff", os.path.join(path, "e2e_ff.csv"))
logger_e2e_ff.info('e2etime_ms')

# linear ff loggers

logger_gpu_compute_linear_ff = setup_logger(path+"_logger_gpu_compute_linear_ff", os.path.join(path, "gpu_compute_linear_ff.csv"))
logger_gpu_compute_linear_ff.info('epoch,gpucomputetime_ms')

logger_epoch_linear_ff = setup_logger(path+"_logger_epoch_linear_ff", os.path.join(path, "epoch_linear_ff.csv"))
logger_epoch_linear_ff.info('epoch_num,epochtime_ms')
"""

'\n# nvidia smi logger\n\nlogger_nvidia_smi = setup_logger(path+"_logger_nvidia_smi", os.path.join(path, "nvidia_smi.csv"))\nsystem_logger = multiprocessing.Process(target=start_logging, args=[logger_nvidia_smi])\n\n# layer loggers\n\nlogger_layer_ff = setup_logger(path+"_logger_layer_ff", os.path.join(path, "layer_ff.csv"))\nlogger_layer_ff.info(\'layer_num,layertime_ms\')\n\n# e2e loggers\n\nlogger_e2e_ff = setup_logger(path+"_logger_e2e_ff", os.path.join(path, "e2e_ff.csv"))\nlogger_e2e_ff.info(\'e2etime_ms\')\n\n# linear ff loggers\n\nlogger_gpu_compute_linear_ff = setup_logger(path+"_logger_gpu_compute_linear_ff", os.path.join(path, "gpu_compute_linear_ff.csv"))\nlogger_gpu_compute_linear_ff.info(\'epoch,gpucomputetime_ms\')\n\nlogger_epoch_linear_ff = setup_logger(path+"_logger_epoch_linear_ff", os.path.join(path, "epoch_linear_ff.csv"))\nlogger_epoch_linear_ff.info(\'epoch_num,epochtime_ms\')\n'

In [9]:
"""
# layer ff timers

layer_ff_start = torch.cuda.Event(enable_timing=True)
layer_ff_end = torch.cuda.Event(enable_timing=True)

# e2e ff timers

e2e_ff_start = torch.cuda.Event(enable_timing=True)
e2e_ff_end = torch.cuda.Event(enable_timing=True)

# linear ff timers

gpu_compute_linear_ff_start = torch.cuda.Event(enable_timing=True)
gpu_compute_linear_ff_end = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_start = torch.cuda.Event(enable_timing=True)
epoch_linear_ff_end = torch.cuda.Event(enable_timing=True)
"""

'\n# layer ff timers\n\nlayer_ff_start = torch.cuda.Event(enable_timing=True)\nlayer_ff_end = torch.cuda.Event(enable_timing=True)\n\n# e2e ff timers\n\ne2e_ff_start = torch.cuda.Event(enable_timing=True)\ne2e_ff_end = torch.cuda.Event(enable_timing=True)\n\n# linear ff timers\n\ngpu_compute_linear_ff_start = torch.cuda.Event(enable_timing=True)\ngpu_compute_linear_ff_end = torch.cuda.Event(enable_timing=True)\nepoch_linear_ff_start = torch.cuda.Event(enable_timing=True)\nepoch_linear_ff_end = torch.cuda.Event(enable_timing=True)\n'

In [10]:
def MNIST_loaders():
    
    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_data = MNIST('./data/', train=True,
              download=True,
              transform=transform)
    test_data = MNIST('./data/', train=False,
              download=True,
              transform=transform)
            
    return train_data, test_data

In [11]:
def FashionMNIST_loaders():

    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_data = FashionMNIST('./data/', train=True,
              download=True,
              transform=transform)

    test_data = FashionMNIST('./data/', train=False,
              download=True,
              transform=transform)

    return train_data, test_data

In [12]:
def CIFAR10_loaders():

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.4914, 0.4822, 0.4465)),
        Lambda(lambda x: torch.flatten(x))])

    train_data = CIFAR10('./data/', train=True,
              download=True,
              transform=transform)

    test_data = CIFAR10('./data/', train=False,
              download=True,
              transform=transform)

    return train_data, test_data

In [13]:
def CIFAR100_loaders():

    transform = Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        Lambda(lambda x: torch.flatten(x))])

    train_data = CIFAR100('./data/', train=True,
              download=True,
              transform=transform)

    test_data = CIFAR100('./data/', train=False,
              download=True,
              transform=transform)

    return train_data, test_data

In [14]:
def SVHN_loaders():

    transform = Compose([
        ToTensor(),
        Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970)),
        Lambda(lambda x: torch.flatten(x))])

    train_data = SVHN('./data/', split='train',
              download=True,
              transform=transform)

    test_data = SVHN('./data/', split='test',
              download=True,
              transform=transform)

    return train_data, test_data

In [15]:
class Net(nn.Module):
    
    def __init__(self, dims, num_labels):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(dims[0], dims[1])
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(dims[1], dims[2])
        self.linear3 = nn.Linear(dims[2], num_labels)
    
    def forward(self,x):
        out1 = self.linear1(x)
        act1 = self.relu(out1)
        out2 = self.linear2(act1)
        act2 = self.relu(out2)
        out3 = self.linear3(act2)
        return out3

In [16]:
net = Net([28*28, 250, 110], num_labels=10)
net

Net(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=250, out_features=110, bias=True)
  (linear3): Linear(in_features=110, out_features=10, bias=True)
)

In [17]:
def train(model, criterion, train_loader, test_loader, optimizer, epochs, input_size):
    for epoch in range(epochs):
        # epoch_linear_ff_start.record() # start recording epoch time
        print(f"Epoch {epoch} of {epochs}")
        train_correct = 0
        total_train_loss = 0
        for i, data in enumerate(tqdm(train_loader)):
            # gpu_compute_linear_ff_start.record() # start recording gpu compute time
            image, label = data
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            
            z = model(image.view(-1, input_size))
            _, prediction = torch.max(z, 1)
            loss = criterion(z, label)
            total_train_loss +=loss.item()
            loss.backward()
            optimizer.step()
            train_correct += (prediction == label).sum().item()
            # gpu_compute_linear_ff_end.record() # end recording gpu compute time
            # torch.cuda.synchronize()
            # logger_gpu_compute_linear_ff.info(str(epoch)+","+str(gpu_compute_linear_ff_start.elapsed_time(gpu_compute_linear_ff_end))) # log gpu compute time
        # epoch_linear_ff_end.record() # end recording epoch time
        # torch.cuda.synchronize()
        # logger_epoch_linear_ff.info(str(epoch)+","+str(epoch_linear_ff_start.elapsed_time(epoch_linear_ff_end))) # log epoch time
        train_error = 100 * (1-(train_correct / len(train_loader.dataset)))
        train_accuracy = 100 * (train_correct / len(train_loader.dataset))
        # print('\nTraining Loss {0}, Accuracy {1:.2f}, Error: {2:.2f}%'.format(total_train_loss/len(train_loader.dataset), train_accuracy, train_error))
    

    train_correct = 0
    total_train_loss = 0
    for i, data in enumerate(train_loader):
        image, label = data 
        image, label = image.to(device), label.to(device)
        z = model(image.view(-1, input_size))
        _, prediction = torch.max(z, 1)
        train_correct += (prediction == label).sum().item()
        loss = criterion(z, label)
        total_train_loss += loss.item()

    train_error = 1-(train_correct / len(train_loader.dataset))
    train_accuracy = train_correct / len(train_loader.dataset)
    print('\nTrain Loss {0}, Accuracy {1:.2f}, Error: {2:.2f}\n'.format(total_train_loss/len(train_loader.dataset), train_accuracy, train_error))

    test_correct = 0
    total_test_loss = 0
    for i, data in enumerate(test_loader):
        image, label = data 
        image, label = image.to(device), label.to(device)
        z = model(image.view(-1, input_size))
        _, prediction = torch.max(z, 1)
        test_correct += (prediction == label).sum().item()
        loss = criterion(z, label)
        total_test_loss += loss.item()

    test_error = 1-(test_correct / len(test_loader.dataset))
    test_accuracy = test_correct / len(test_loader.dataset)
    print('\nTest Loss {0}, Accuracy {1:.2f}, Error: {2:.2f}\n'.format(total_test_loss/len(test_loader.dataset), test_accuracy,test_error))
    
    return train_accuracy, test_accuracy

In [18]:

if __name__ == "__main__":
    reference_time = time.time()

    torch.manual_seed(1234)
    
    if dataset == 'MNIST':
      train_data, test_data = MNIST_loaders()
    elif dataset == 'FashionMNIST':
      train_data, test_data = FashionMNIST_loaders()
    elif dataset == 'CIFAR10': 
      train_data, test_data = CIFAR10_loaders()
    elif dataset == 'CIFAR100':
      train_data, test_data = CIFAR100_loaders()
    elif dataset == 'SVHN':
      train_data, test_data = SVHN_loaders()
    else:
      print('Invalid dataset')
      exit()
    print(train_data, test_data)
    num_train = 50000
    num_test = 10000
    subsample_train_indices = torch.randperm(len(train_data))[:num_train]
    subsample_test_indices = torch.randperm(len(test_data))[:num_test]
    train_loader = DataLoader(train_data, batch_size=256, shuffle=False, sampler=SubsetRandomSampler(subsample_train_indices))
    test_loader = DataLoader(test_data, batch_size=256, shuffle=False, sampler=SubsetRandomSampler(subsample_test_indices))
    print(len(train_loader.dataset))
    
    if dataset == 'MNIST' or dataset == 'FashionMNIST':
      input_size = 784
    elif dataset == 'SVHN' or dataset == 'CIFAR10' or dataset == 'CIFAR100':
      input_size = 3072
    
    if dataset == 'MNIST' or dataset == 'FashionMNIST':
      model = Net([input_size, 250, 110], num_labels=10).to(device) # MNIST and FashionMNIST
      epochs = 20
    elif dataset == 'CIFAR10':
      model = Net([input_size, 250, 110], num_labels=10).to(device) # CIFAR10, CIFAR100
      epochs = 20
    elif dataset == 'CIFAR100':
      model = Net([input_size, 250, 110], num_labels=100).to(device)
      epochs = 20
    elif dataset == 'SVHN':
      model = Net([input_size, 75, 50], num_labels=10).to(device) # SVHN
      epochs = 20
    else:
      print('Invalid dataset')
      exit()
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.00067
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # system_logger.start()
    # e2e_ff_start.record() # start recording e2e train time
    train_accuracy, test_accuracy = train(model, criterion, train_loader, test_loader, optimizer, epochs, input_size)
    # e2e_ff_end.record() # end recording e2e train time
    # system_logger.terminate()
    # torch.cuda.synchronize()
    # logger_e2e_ff.info(str(e2e_ff_start.elapsed_time(e2e_ff_end))) # log e2e train time
    print('Dataset:', dataset)
    print('# Train Samples:', len(subsample_train_indices), ', # Test Samples:', len(subsample_test_indices))
    print('Train Accuracy:', train_accuracy)
    print('Test Accuracy:', test_accuracy)

100%|██████████| 170498071/170498071 [00:13<00:00, 13005126.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data/
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.4914, 0.4822, 0.4465))
               Lambda()
           ) Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.4914, 0.4822, 0.4465))
               Lambda()
           )
50000
Epoch 0 of 20


100%|██████████| 196/196 [00:17<00:00, 11.08it/s]


Epoch 1 of 20


100%|██████████| 196/196 [00:15<00:00, 12.92it/s]


...

Epoch 18 of 20


100%|██████████| 196/196 [00:15<00:00, 12.43it/s]


Epoch 19 of 20


100%|██████████| 196/196 [00:15<00:00, 12.55it/s]



Train Loss 0.001887649183869362, Accuracy 0.84, Error: 0.16


Test Loss 0.006746766567230225, Accuracy 0.53, Error: 0.47

Dataset: CIFAR10
# Train Samples: 50000 , # Test Samples: 10000
Train Accuracy: 0.83818
Test Accuracy: 0.5298


In [ ]:
# !zip -r /content/backprop.zip /content/backprop/